In [1]:

import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.optimize import curve_fit
import time


# Start timer
start_time = time.perf_counter()


In [2]:
#!pip install modestga --upgrade --quiet
#!pip install deap --upgrade --quiet
import warnings
warnings.filterwarnings("ignore")

In [3]:
#===========================================================
#                   Read Excel File (Node_Property)
#===========================================================

file = pd.ExcelFile('Nodes-Property.xlsx')
df1=pd.read_excel(file,'Node_Property')
data=df1[df1.columns[:]].to_numpy()
data = np.unique(data,axis=0)
#print ('data: \n',data)
Node_Name= data[:,0]
Memory= data[:,1]                                # Accessible memory size of CNk for running tasks
tsk= data[:,2]                                   # Switch time between two jobs for CNk
#print(tsk)
#===========================================================
#                   Read Excel File
#===========================================================

file = pd.ExcelFile('results_bacteria30g_4nodes.xlsx')
df1=pd.read_excel(file,'General-Results')
data=df1[df1.columns[:]].to_numpy()
data = np.unique(data,axis=0)
#print ('data: \n',data)
nodename= data[:,0]
x= data[:,1]
y= data[:,2]


nodename_item =list(dict.fromkeys(nodename))
V = len(nodename_item)  # The number of nodes of the cluster

#===========================================================
#                   Read Excel File
#===========================================================
#genomes_df= pd.read_csv('overview-test.xlsx')
genomes_df= pd.read_excel('overview_results_bacteria30g.xlsx')


#===========================================================
#                   Initializing Variables
#===========================================================

#CNk=0                             # kth node of the cluster
#Memk=0                            # Accessible memory size of
                                   # Switch time between two jobs for CNk
n=len(genomes_df.GenomeName)       # The number of genomes of the reference genome
Gen= genomes_df.GenomeName         #
Gen= Gen[0:n]                      # The set of genomes of the reference genome
Si=genomes_df.Size                 #
Si= Si[0:n]                       # Size of gi
#J = n                           # The maximum number of Jobs
J = V
poly1=[]
poly2=[]
poly3=[]
#poly4=[]


Dictionary={}
FinalOutput={}
#===========================================================
#                   Initializing Genetic Algorithm parameters
#===========================================================

solution =[]

population_number=30
MutationRate= 0.4
CrossoverRate= 1 - MutationRate
#maxIter = 120
maxIter = 20
MutationNumber = round (population_number* MutationRate);
CrossoverNumber = round (population_number * CrossoverRate);


In [4]:
#================================================================================================
#                  f(X)= aX+b
#================================================================================================

def f1 (x, a, b):
    t=(x*a+b)
    return t


#================================================================================================
#                  f(X)= aX^2 + bX + c
#================================================================================================

def f2 (x, a,b,c):
    t=(x**2*a +x*b+c)
    return t



#================================================================================================
#                  f(X)= aX^3 + bX^2 + cX + d
#================================================================================================
def f3 (x, a,b,c,d):
    t= x**3*a +x**2*b +x*c+d
    return t


#================================================================================================
#                  f(X)= aX^4 + bX^3+ cX^2 + dX + e
#================================================================================================
'''def f4 (x, coefficient):
    t=(x**4*coefficient[0] +x**3*coefficient[1] +x**2*coefficient[2] +x*coefficient[3]+coefficient[4])
    return t  '''
#================================================================================================
#                  fk(X)
#================================================================================================
def fk (k,x, coefficients):
    if k==1:
        return f1(x, *coefficients)
    if k==2:
         return f2(x, *coefficients)
    if k==3:
         return f3(x, *coefficients)
    #if k==4:
         #return f4(x, coefficients)


#================================================================================================
#                  Find The Best or Simplest Polynomaial Function Based on Mean Square Error
#================================================================================================

Estimated_value_by_degree_1 =[]
Estimated_value_by_degree_2 =[]
Estimated_value_by_degree_3 =[]
#Estimated_value_by_degree_4 =[]
MSE=[]
BestPloynomialDegree=[[]]
for i in range (V):
  MSE=[]
  #print('i : ', i)
  a= data ==nodename_item[i]
  nodenamep= nodename[a[:,0]]
  xp= x[a[:,0]]
  yp= y[a[:,0]]
  #print('Original X for node name of {} is: {}'.format(int(nodenamep[0]),xp))
  #print('Original Y for node name of {} is: {}'.format(int(nodenamep[0]),yp))
  #print('\n')

  #======================================================
  # Polynomaial Equation Degree of 1  & MSE
  #======================================================
  #coefficient1= np.polyfit(xp,yp,1);
  coefficient1, covariance1 = curve_fit(f1, xp, yp,bounds=(0, [99999., 999999.]))
  poly1.append(coefficient1)
  #print('coefficient1= ',coefficient1)
  Estimated_y_by_degree_1 = f1(xp,*coefficient1);
  #print('Estimated Y by Degree of 1= ',  Estimated_y_by_degree_1)
  MSE.append(mean_squared_error(yp,Estimated_y_by_degree_1))


  #======================================================
  # Polynomaial Equation Degree of 2 & MSE
  #======================================================
  if (len(xp)>1):
      #coefficient2= np.polyfit(xp,yp,2);
      coefficient2, covariance2 = curve_fit(f2, xp, yp,bounds=(0, [99999., 999999.,999999. ]))
      poly2.append(coefficient2)
      #print('coefficient2= ',coefficient2)
      Estimated_y_by_degree_2=f2(xp,*coefficient2);
      #print('Estimated Y by Degree of 2= ', Estimated_y_by_degree_2)
      MSE.append( mean_squared_error(yp,Estimated_y_by_degree_2))


  #======================================================
  # Polynomaial Equation Degree of 3 & MSE
  #======================================================
  if (len(xp)>2):
      #coefficient3= np.polyfit(xp,yp,3)
      coefficient3, covariance3 = curve_fit(f3, xp, yp ,bounds=(0, [99999., 999999.,999999. ,999999. ]))
      poly3.append(coefficient3)
      #print('coefficient3= ',coefficient3);
      Estimated_y_by_degree_3=f3(xp,*coefficient3)
      #print('Estimated Y by Degree of 3= ',Estimated_y_by_degree_3 )
      MSE.append( mean_squared_error(yp,Estimated_y_by_degree_3))


  #======================================================
  # Polynomaial Equation Degree of 4 & MSE
  #======================================================
  ''' if (len(xp)>3):
      coefficient4= np.polyfit(xp,yp,4)
      poly4.append(coefficient4);
      #print('coefficient4= ',coefficient4)
      Estimated_y_by_degree_4= f4(xp,coefficient4)
      #print('Estimated Y by Degree of 4= ',Estimated_y_by_degree_4)
      MSE.append( mean_squared_error(yp,Estimated_y_by_degree_4))  '''


  #======================================================
  # Find Minimum of MSE and its index
  #======================================================
  #print('\n')
  #print ("MSE = ", MSE)
  MinMSE =min(MSE)
  #print("Min(MSE) = ", MinMSE)
  Dictionary={}
  Dictionary["NodeName"] = int( nodenamep[0])
  Dictionary["BestPloynomialDegree"]=MSE.index(MinMSE)+1
  if (MSE.index(MinMSE)==0):
      Dictionary["Coefficients"]=coefficient1
  elif (MSE.index(MinMSE)==1):
      Dictionary["Coefficients"]=coefficient2
  elif (MSE.index(MinMSE)==2):
      Dictionary["Coefficients"]=coefficient3
  #elif (MSE.index(MinMSE)==3):
     # Dictionary["Coefficients"]=coefficient4

  FinalOutput[i]= Dictionary
  BestPloynomialDegree.append([nodenamep[0],MSE.index(MinMSE)+1])

  #print('The Best and simplest Ploynomial Degree for node name {} is: {}'.format(int(nodenamep[0]),MSE.index(MinMSE)+1))
  #print('\n')
  #print ('*****************************************************************************************')
  #print('\n')
#======================================================
# Print result for Best Polynomaial Equation Degree Based MSE
#======================================================
#print('\n')
#print('==========================================================================================')
#print('           Best Ploynomial Degree (Based on MSE): \n', BestPloynomialDegree)
#print('==========================================================================================')
#print('\n')
#print('\n')
#print('\n')

FinalOutput






{0: {'NodeName': 213,
  'BestPloynomialDegree': 2,
  'Coefficients': array([1.55991657e-04, 4.74452689e-01, 9.96033867e+01])},
 1: {'NodeName': 219,
  'BestPloynomialDegree': 1,
  'Coefficients': array([  0.47855627, 154.10289488])},
 2: {'NodeName': 223,
  'BestPloynomialDegree': 1,
  'Coefficients': array([  0.34702   , 319.18664096])},
 3: {'NodeName': 228,
  'BestPloynomialDegree': 3,
  'Coefficients': array([5.03274153e-08, 3.08948728e-20, 4.60596094e-01, 1.92132060e+02])}}

In [5]:
#Compute job size= [7570.300000000008, 7776.800000000008, 7694.699999999989, 7650.699999999992]
for i in range (V):
    print(i,fk(FinalOutput[i]['BestPloynomialDegree'],7650.699999999992,FinalOutput[i]['Coefficients']))


0 12860.191083928237
1 3815.393312261109
2 2974.1325388055798
3 26253.639420897616


In [6]:
def jobs2cluster(S):
    AssignedJobs2Cluster={}
    for i in range (V):
        AssignedJobs2Cluster[i]=[]
    for j in range (n):
        Jobj=int(S[0][j])
        CNk=int(S[1][j])
        AssignedJobs2Cluster[CNk].append(Jobj)
    return (AssignedJobs2Cluster)

In [7]:
def genome2cluster(SS):
    g2c={}
    for i in range (V):
        g2c[i]=[]
    for j in range (n):
        Genomej=j
        CNk=int(SS[1][j])
        g2c[CNk].append(Genomej)
    return(g2c.copy())

In [8]:
"""Feasibility function for the Memk. Returns True if feasible False
    otherwise."""
def feasibleMemk(sol):


    M=np.zeros((V,J))



    for i in range (n):
        jobj= int(sol[0][i])
        CNk = int(sol[1][i])

        item= Si[i]
        q= len(item)
        if item[q-1]== 'K' or item[q-1]== 'k' :
            DigitSize=item[:-1]

            M[CNk][jobj]+= (float(DigitSize)/1000)
        elif  item[q-1]== 'M' or item[q-1]== 'm' :
            DigitSize=item[:-1]
            M[CNk][jobj]+= (float(DigitSize))

    for k in range (V):
        for j in range (J):
            if  Memory[k]< 3*M[k][j]:
                return False
    return True



In [9]:
def ComputeJobSize(g2j):
    sm=[]
    for j in range (J):
            sm.append(0.0)
    for j in range (J):

            GenomesNumber= g2j[j]
            m= len(GenomesNumber)
            #print(" GenomesNumber:",  GenomesNumber)

            #extract memory size of each genome belonged to Job j
            MemoryItems = (Si[GenomesNumber])
            #print (' MemoryItems :',  MemoryItems )
            for i in GenomesNumber:
                item= Si[i]
                #print ('Genome Size {} is: {}'.format(i,Si[i]))
                q= len(item)
                if item[q-2]== 'K' or item[q-2]== 'k' :
                    DigitSize=item[:-1]
                    sm[j]+= float(DigitSize)/1000
                elif  item[q-2]== 'M' or item[q-2]== 'm' :
                    DigitSize=item[:-2]
                    sm[j]+= float(DigitSize)
    print('Compute job size=',sm.copy() )
    return sm.copy()

In [10]:
'''def RunningTime (Sl,g2cn):
    tk= np.zeros((V,1))
    J2C=jobs2cluster(Sl)
    G2J=Genomes2Jobs(Sl)
    JS= ComputeJobSize(G2J) # Job Size
    for k in range (V):
        NeedMemory=0
        for j in range (len(np.unique(J2C[k]))):
            #print ('Cluster Number is: {},  Job Number is {}, and Job size is {}. \n '.format (k,J2C[k][j],JS[J2C[k][j]]))
            NeedMemory=JS[J2C[k][j]]
            if(NeedMemory == 0) :
                print("error 0")
                continue
            else:
                tk[k]+=fk(FinalOutput[k]['BestPloynomialDegree'],NeedMemory,FinalOutput[k]['Coefficients'])
    for i in range (V):
        GenomesNumber= g2cn[i]
        m= len(np.unique(J2C[i]))
        tk[i]+= ((m-1)*tsk[i])
    print('tk=',max(tk))
    return max(tk)
'''
def RunningTime (Sl,g2cn):
    tk= np.zeros((V,1))
    J2C=jobs2cluster(Sl)
    G2J=Genomes2Jobs(Sl)
    JS= ComputeJobSize(G2J) # Job Size
    for k in range (V):
        NeedMemory=0
        for j in range (len(np.unique(J2C[k]))):
            #print ('Cluster Number is: {},  Job Number is {}, and Job size is {}. \n '.format (k,J2C[k][j],JS[J2C[k][j]]))
            NeedMemory=JS[J2C[k][j]]
            if(NeedMemory == 0) :
                print("error 0")
                continue
            else:
                tk[k]+=fk(FinalOutput[k]['BestPloynomialDegree'],NeedMemory,FinalOutput[k]['Coefficients'])
        #print(' #jobs assigned to Node[',k,']=',J2C[k])
        print('tk[',k,']=',tk[k])
    for i in range (V):
        GenomesNumber= g2cn[i]
        m= len(np.unique(J2C[i]))
        tk[i]+= ((m-1)*tsk[i])
    #if (max(tk)== -10):
    print('tk=',max(tk))
    return max(tk)





In [11]:
def Genomes2Jobs(SL):
    Ge2Jo={}
    for i in range (n):
        Ge2Jo[i]=[]
    for j in range (n):
        Genomej=j
        Jobj=int(SL[0][j])
        Ge2Jo[Jobj].append(Genomej)
    return(Ge2Jo.copy())

In [12]:
def clusters2job(solu):
    C2Job={}
    for i in range (n):
        C2Job[i]=[]
    for j in range (n):
        Jobj=int(solu[0][j])
        CNk=int(solu[1][j])
        C2Job[Jobj].append(CNk)

    return(C2Job.copy())


In [13]:
import random
def AddressJobs2ClustersProblem(SOL):
    Temp= clusters2job(SOL.copy())
    t= len(Temp)
    for j in range (len(Temp)):
        if len(Temp[j])>1:
            newCNkIndex= random.randint(0,len(Temp[j])-1)
            newCNk= Temp[j][newCNkIndex]
            for i in range (n):
                if SOL[0][i]== j:
                    SOL[1][i]=newCNk
    return SOL.copy()








In [14]:
def equalSols(S1,S2):
    for j in range(len(S1)):
        if (S1[1][j]!= S2[1][j] or S1[0][j]!= S2[0][j] ):
                return False
    return True

In [15]:
def isRepeated(POp,Sol):
    index= False
    newPop=[]
    for i in range(len(POp)):
        newPop.append((POp[i].copy()))
    newPop=np.array(newPop.copy())
    N= len(POp)
    for i in range(N):
        if (equalSols(Sol.copy(),newPop[i,:,:].copy())):
                index =True
    return index

In [16]:
def arrangeSolutions(POPU):
    newFit = []
    newPop=np.empty((len(POPU), 2,n))
    for i in range(len(POPU)):
         if (feasibleMemk(POPU[i].copy())==True):

            newPop[i]=((POPU[i].copy()))
            SolvedSol= AddressJobs2ClustersProblem(POPU[i].copy())
            G2CNK= genome2cluster(SolvedSol.copy())
            Fit= RunningTime(SolvedSol.copy(),G2CNK)
            newFit.append(Fit)

    return newPop.copy(),newFit.copy()



In [17]:
def AddNewSol(POPulation,Sol,FITS,Fit):
    index= isRepeated(POPulation.copy(),Sol.copy())
    L= len( POPulation.copy())
    if(index == False ):
        L=L+1
        POPulation= np.append(POPulation.copy(),Sol.copy())
        POPulation= POPulation.reshape(L,2,n)
        FITS.append(Fit)

    return POPulation.copy(),FITS.copy()

In [18]:
def SortPopulation(poP, FS,Npop):

    newFit = []
    newPop=[]

    for i in range(len(poP)):
        newPop.append((poP[i].copy()))
        newFit.append(FS[i][0].copy())
    s = np.array(newFit.copy())

    sort_index = np.argsort(newFit.copy())
    for i in range(len(poP)):
        newPop[i]=list( poP[sort_index[i]].copy())
    FS.sort()

    return newPop.copy(),FS.copy(),len(newPop.copy())

In [19]:

def CreatePopulation (n):
    #population[i][j][k] ==> i th population
    #[row0] ==> Job of solution [.......]
    #[row1] ==> CNk of solution [.......]
    Npop=1
    NS= np.zeros((2,n))
    for i in range (n):
            NS[0,i]=int(random.randint(0,J-1))# Random Job  number
            NS[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
    NS= AddressJobs2ClustersProblem(NS.copy())
    Sol= genome2cluster(NS.copy())

    #JobSize = totalMemk(Sol)
    if (feasibleMemk(NS.copy())==True):
            POPULATION= np.empty((1, 2,n))
            for i in range(Npop):
                 POPULATION[0]= NS.copy()
            Fit= RunningTime(NS.copy(),Sol)
    else:
            while(feasibleMemk(NS.copy())==False):
                NS= np.zeros((2,n))
                for i in range (n):
                    NS[0,i]=int(random.randint(0,J-1))# Random Job  number
                    NS[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
                NS= AddressJobs2ClustersProblem(NS.copy())

                Sol= genome2cluster(NS.copy())

                #JobSize = totalMemk(Sol)


                if (feasibleMemk(NS.copy())==True):

                    POPULATION= np.empty((1, 2,n))
                    for j in range(n):
                        POPULATION[0]= NS.copy()
                    Fit= RunningTime(NS.copy(),Sol)

    return POPULATION.copy(),Fit.copy()



In [20]:
def Mutation(PP,FITS):
    P=PP.copy()
    FF=FITS.copy()
    for iMut in range(MutationNumber):
            Npop= len(PP)
            SelectedSolutionIndex= random.randint(0,len(P)-1)
            SelectedSolution= (P[SelectedSolutionIndex]).copy()
            #print("SelectedSolution:\n", SelectedSolution)

            SelectedGenomeIndex1 = random.randint(0,n-1)
            SelectedGenomeIndex2 = random.randint(0,n-1)

            tempJ=SelectedSolution[0][SelectedGenomeIndex1]
            tempCNk=SelectedSolution[1][SelectedGenomeIndex1]
            SelectedSolution[0][SelectedGenomeIndex1]= SelectedSolution[0][SelectedGenomeIndex2].copy()
            SelectedSolution[1][SelectedGenomeIndex1]= SelectedSolution[1][SelectedGenomeIndex2].copy()
            SelectedSolution[0][SelectedGenomeIndex2]= tempJ
            SelectedSolution[1][SelectedGenomeIndex2]= tempCNk
            MutedSolution = AddressJobs2ClustersProblem(SelectedSolution.copy())
            Sol= genome2cluster(MutedSolution.copy())




            if (feasibleMemk(MutedSolution.copy())==True and  isRepeated(P.copy(),MutedSolution.copy()) ==False):
                #print('Mut len:',len(MutedSolution))
                F=( RunningTime(MutedSolution.copy(),Sol.copy()))
                P, FF= AddNewSol(P.copy(), MutedSolution.copy(),FITS.copy(),F)
                if len(P)!= len(FF):
                    P,F= arrangeSolutions(P)
                    #print("Pop:\n", PP)
                    #print("!!!Mutelen(P):",len(P))
                    #print ("len Fit",len(F))
    return P, FF



In [21]:
def Crossover (POP,FITN):
    PoP=POP.copy()
    FiT=FITN.copy()
    for iCross in range(CrossoverNumber):
            Npop= len(PoP)
            SelectedSolutionIndex1= random.randint(0,len(PoP)-1)
            SelectedSolutionIndex2= random.randint(0,len(PoP)-1)

            SS1= PoP[SelectedSolutionIndex1].copy()
            SS2= PoP[SelectedSolutionIndex2].copy()

            SelectedGenomeIndex = random.randint(0,n-1)
            temp=  SS2[SelectedGenomeIndex:n-1].copy()
            SS2[SelectedGenomeIndex:n-1]= SS1[SelectedGenomeIndex:n-1].copy()
            SS1[SelectedGenomeIndex:n-1]= temp.copy()


            SS1=AddressJobs2ClustersProblem(SS1.copy())
            Sol1= genome2cluster(SS1.copy())



            #JobSize = totalMemk(Sol1)
            if (feasibleMemk(SS1)==True and isRepeated(PoP.copy(),SS1.copy()) ==False):
                #print('Cross1 len:',len(SS1))
                F=(RunningTime(SS1.copy(),Sol1.copy()))
                PoP,FiT= AddNewSol(PoP.copy(), SS1.copy(),FiT.copy(),F)



            SS2=AddressJobs2ClustersProblem(SS2)
            Sol2= genome2cluster(SS2)
            #JobSize = totalMemk(Sol2)
            if (feasibleMemk(SS2.copy())==True and isRepeated(PoP.copy(),SS2.copy()) ==False):
                #print('Croos2 len:',len(SS2))
                F=(RunningTime(SS2.copy(),Sol2.copy()))
                PoP,FiT= AddNewSol(PoP.copy(), SS2.copy(),FiT.copy(),F)
    if (len(PoP)!= len(FiT)):
        PoP,FiT= arrangeSolutions(PoP.copy())
        #print("Pop:\n", PP)
        #print("!!!Crosslen(P):",len(PoP))
        #print ("len Fit",len(FiT))
    return PoP.copy(),FiT.copy()

In [22]:
#==================================================================
#                 Genetic Algorithm
#==================================================================

def GA_Scheduling():
    #==================================================================
    #                 Create Initial Population
    #===================================================================
    Population=[]
    #MaxPopN= 1000
    MaxPopN= 10

    Npop=0  # number of added acceptable solutions to population
    fits=[]

    Population,Fit= CreatePopulation (n)
    fits.append(Fit)
    #print('initial Population',Population)
    while (Npop <population_number):
        for j in range (population_number):
            NSOL= np.zeros((2,n))
            for i in range (n):
                NSOL[0,i]=int(random.randint(0,J-1))# Random Job  number
                NSOL[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
            NSOL= AddressJobs2ClustersProblem(NSOL)

            Sol= genome2cluster(NSOL)
            #JobSize = totalMemk(Sol)
            if (feasibleMemk(NSOL)==True and isRepeated(Population.copy(),NSOL.copy()) ==False):
                Fit= (RunningTime(NSOL,Sol))
                Population, fits= AddNewSol(Population.copy(),NSOL.copy(),fits.copy(), Fit)
                Npop+=1
    #print('initpop:',Population)

    #==================================================================
    #                 Genetic Algorithm
    #===================================================================
    for iteration in range(maxIter):
        changed=False

        changed=False
        nMut=0   # number of acceptable Muted solutions
        nCross=0 # number of acceptable Crossover solutions
        Npop= len(Population)

        #for i in range (len (Population)):
            #print(Population[i])
        #==================================================================
        #                 Mutation
        #===================================================================

        #print ('Mut1:')
        #print(len(Population))
        #print(len(fits))
        Population,fits= Mutation(Population.copy(),fits.copy())
        #print ('Mut2:')
        #print(len(Population))
        #print(len(fits))
        #for i in range (len (Population)):
            #print(Population[i])
        #==================================================================
        #                Crossover
        #===================================================================
        #print ('Cros1:')
        #print(len(Population))
        #print(len(fits))
        Population,fits=Crossover (Population.copy(),fits.copy())
        #print ('Cros2:')
        #print(len(Population))
        #print(len(fits))

        #==================================================================
        #                Add Muted Solutions to Population
        #===================================================================

        ''' print ('Pop1:')
        for i in range (len (Population)):
            print(Population[i])'''
        Population, fits,Npop=SortPopulation(Population.copy(), fits.copy(), Npop-1)
        Population,fits= arrangeSolutions(Population.copy())

        '''print ('Pop2:')
        for i in range (len (Population)):
            print(Population[i])'''
        '''print("\n***************************************************************************\n")
        print('Best Solution is of iteration of {} is:\n{} '.format(iteration,Population[0].copy() ))
        print("-----------------------------------------------------------------------------\n") '''

    return Population.copy(), fits.copy()

P, F= GA_Scheduling()

Compute job size= [7783.900000000003, 7490.89999999999, 7647.299999999997, 7770.400000000009]
tk[ 0 ]= [0.]
tk[ 1 ]= [0.]
tk[ 2 ]= [3015.67083255]
tk[ 3 ]= [75816.23809702]
tk= [75848.23809702]
Compute job size= [7557.699999999995, 7749.000000000008, 7814.000000000002, 7571.79999999999]
tk[ 0 ]= [13331.60827902]
tk[ 1 ]= [3770.88757961]
tk[ 2 ]= [2946.75266097]
tk[ 3 ]= [27178.84946756]
tk= [27178.84946756]
Compute job size= [7565.399999999994, 7781.899999999996, 7679.5000000000055, 7665.700000000004]
tk[ 0 ]= [12617.25349837]
tk[ 1 ]= [7707.35562694]
tk[ 2 ]= [0.]
tk[ 3 ]= [26393.37056795]
tk= [26393.37056795]
Compute job size= [7825.5000000000055, 7662.099999999993, 7589.199999999998, 7615.700000000005]
tk[ 0 ]= [12892.83067175]
tk[ 1 ]= [3798.64384298]
tk[ 2 ]= [2952.79080894]
tk[ 3 ]= [27914.50588449]
tk= [27914.50588449]
Compute job size= [7802.600000000013, 7457.30000000001, 7977.699999999984, 7454.899999999993]
tk[ 0 ]= [13298.42844217]
tk[ 1 ]= [7693.5732065]
tk[ 2 ]= [2907.018

In [23]:
BestSolution = P[0]
ClusterName= np.unique(nodename)
print('The best solution:\n')
for i in range(n):
    j= int(BestSolution[0][i])
    k= int(BestSolution[1][i])
    #print( "Genome {} is allocated to Job Number {} on Cluster Node Number {}.".format(i,j,ClusterName[k]))

print('\n and its fitness function is:\n',(F[0]))

The best solution:


 and its fitness function is:
 [7584.9770277]


In [24]:
'''S1=P[0]
print(feasibleMemk(S1))

print(S1)
'''

'S1=P[0]\nprint(feasibleMemk(S1))\n\nprint(S1)\n'

In [25]:

J2C= jobs2cluster(BestSolution)
EmptyClusters=[]
for k in range(V):
    if (J2C[k]!= []):
        print('Cluster {} includes a subset of Job numbers:\n{}'.format(ClusterName[k],np.unique( J2C[k])))
    else:
        EmptyClusters.append(ClusterName[k])
print("================================================================================")
print ("Empty Clusters are:", (EmptyClusters))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

#print('Size Job number{} = {}'.format(np.unique( J2C[k]), )


Cluster 219.0 includes a subset of Job numbers:
[0 2]
Cluster 223.0 includes a subset of Job numbers:
[1 3]
Empty Clusters are: [213.0, 228.0]
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


In [26]:
def NewComputeJobSize(g2j):
    sm=[]
    for j in range (J):
            sm.append(0.0)
    for j in range (J):

            GenomesNumber= g2j[j]
            m= len(GenomesNumber)
            #print(" GenomesNumber:",  GenomesNumber)

            #extract memory size of each genome belonged to Job j
            MemoryItems = (Si[GenomesNumber])
            #print (' MemoryItems :',  MemoryItems )
            for i in GenomesNumber:
                item= Si[i]
                #print ('Genome Size {} is: {}'.format(i,Si[i]))
                q= len(item)
                if item[q-2]== 'K' or item[q-2]== 'k' :
                    DigitSize=item[:-1]
                    sm[j]+= float(DigitSize)/1000
                elif  item[q-2]== 'M' or item[q-2]== 'm' :
                    DigitSize=item[:-2]
                    sm[j]+= float(DigitSize)
    print('Compute job size=',sm.copy() )
    return sm.copy()

In [27]:
G2J= Genomes2Jobs(BestSolution)
JobSize=NewComputeJobSize(G2J)
EmptyJobs=[]
for i in range(J):
    if (G2J[i]!= []):
        print('Job {} includes a subset of genomes numbers:{}'.format(i,G2J[i]))
    else:
        EmptyJobs.append(i)
print("================================================================================")
print ("Empty Jobs are:", EmptyJobs)
print ("The number of Empty Jobs is:", len(EmptyJobs))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
for k in range(J):
    print('Size Job number{} = {}'.format(k,JobSize[k]))
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print('running time of the best solution=' )

Compute job size= [7533.900000000002, 7602.499999999992, 7590.300000000007, 7965.800000000003]
Job 0 includes a subset of genomes numbers:[4, 6, 7, 12, 16, 20, 24, 26, 27, 29, 31, 32, 45, 56, 58, 62, 65, 70, 72, 73, 74, 75, 77, 82, 84, 86, 91, 95, 96, 97, 102, 104, 108, 109, 112, 116, 125, 127, 132, 135, 142, 153, 154, 155, 157, 162, 164, 165, 176, 209, 211, 221, 226, 228, 229, 231, 232, 233, 238, 240, 241, 242, 244, 254, 256, 261, 264, 266, 269, 270, 272, 275, 283, 288, 292, 293, 297, 298, 306, 313, 319, 320, 322, 323, 324, 326, 328, 329, 330, 333, 337, 340, 342, 352, 353, 360, 379, 383, 391, 396, 406, 409, 412, 413, 421, 424, 429, 438, 439, 442, 443, 444, 445, 450, 452, 455, 459, 464, 465, 470, 474, 486, 494, 502, 505, 506, 508, 509, 510, 511, 512, 519, 520, 525, 532, 533, 535, 536, 539, 540, 547, 549, 551, 561, 562, 565, 569, 571, 572, 577, 578, 582, 585, 588, 592, 597, 602, 605, 606, 609, 612, 616, 618, 623, 628, 639, 653, 654, 655, 661, 669, 674, 681, 683, 693, 703, 707, 709, 716,

In [28]:
#print(len(F))
#print(len(P))
#for i in range(len(P)):
 #   print('Jobs and for Population {} is:\t{}, {} fits={}'.format(i,P[i][0],P[i][1],F[i]))

#

In [29]:
# End timer
end_time = time.perf_counter()
print('\n and its fitness function is:\n',int(F[0]))
print("================================================================================")
# Calculate elapsed time
elapsed_time = end_time - start_time
print("Elapsed time: ", int(elapsed_time))
print("================================================================================")
print ("Empty Clusters are:", len(EmptyClusters))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print ("The number of Empty Jobs is:", len(EmptyJobs))


 and its fitness function is:
 7584
Elapsed time:  76
Empty Clusters are: 2
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
The number of Empty Jobs is: 0
